In [ ]:
import torch
import pandas as pd
import os
import numpy as np
from numpy.random.mtrand import random_integers
from scipy.sparse import rand
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch.nn.functional as F
# Setup - you have to ensure this path is correct for your google drive too
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Neuro Project/Rat Neural Data/Valid Trials Data'
os.listdir(folder_path)

##########DEFINE NN AND FUNCTIONS:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

def poisson_loss(y_pred, y_true):
    # Ensuring non-negative predictions
    y_pred = torch.clamp(y_pred, min=0, max=float('inf'))
    # Adding epsilon to avoid log(0)
    return torch.mean(y_pred - y_true * torch.log(y_pred + 1e-8))

#specify the network structure:
# create model
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2,hidden_size3,hidden_size4,output_size):
        super(Net, self).__init__()

        self.input_size = input_size #400
        self.hidden_size1 = hidden_size1 #?? 300
        self.hidden_size2 = hidden_size2 #?? 200
        self.hidden_size3 = hidden_size3 #?? 100
        self.hidden_size4 = hidden_size4 #?? 50
        self.output_size = output_size #?? 1

        self.iToh1 = torch.nn.Linear(input_size, hidden_size1) #400,300
        self.h1Toh2 = torch.nn.Linear(hidden_size1, hidden_size2) #300,200
        self.h2Toh3 = torch.nn.Linear(hidden_size2, hidden_size3) #200,100
        self.h3Toh4 = torch.nn.Linear(hidden_size3, hidden_size4) #100,50
        self.h4ToO = torch.nn.Linear(hidden_size4, output_size) #50,1
        self.sigmoid = torch.nn.Sigmoid()
        #self.Conv1d = torch.nn.Conv1d(in_channels=13, out_channels=1, kernel_size=1) ##This in concert with other changes will let you select random subset of trials.
        self.ReLU = torch.nn.ReLU()
        self.LSM = torch.nn.LogSoftmax(dim=1)

    def forward(self, x):
        #a = self.Conv1d(x)
        x = self.LSM(self.iToh1(x)) ###was LSM
        x = self.LSM(self.h1Toh2(x)) ##was sigmoid
        x = self.LSM(self.h2Toh3(x)) ##was sigmoid
        x = self.LSM(self.h3Toh4(x)) ##was sigmoid
        x = self.LSM(self.h4ToO(x)) ##was sigmoid #no activation here if using x-entropy loss, bc already has one
        return x ###maybe restrict to examples with over 30 neurons.

###########DO THE LEARNING
PossibleFiles = ['R397-2017-06-09', 'R417-2017-10-28','R417-2017-10-31','R397-2017-06-09', 'R417-2017-10-28','R417-2017-10-31','R417-2017-11-02','R423-2017-08-02','R423-2017-08-04','R423-2017-08-05','R423-2017-08-07','R442-2018-03-17','R442-2018-03-19', 'R442-2018-03-20', 'R442-2018-03-22','R465-2018-08-14','R465-2018-08-15','R465-2018-08-18','R465-2018-08-20','R497-2018-11-01','R497-2018-11-04','R497-2018-11-05','R501-2019-01-04','R501-2019-01-06','R501-2019-01-09','R522-2019-03-13','R522-2019-03-15','R522-2019-03-16','R522-2019-03-18'];
FileSets = np.random.choice(len(PossibleFiles), len(PossibleFiles), replace = False)
TrainVal = FileSets[0:18] #training examples...~60/40 split.
TestVal =  FileSets[18:30]

for i in range(0,len(TrainVal)-1): #reserve last file in TrainVal for validation.
  # For the regular data, the goal is just to get it into vector form.
  Target_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TrainVal[i]]+"_KD.csv"))
  Target_Tensor = torch.tensor(Target_df.values)
  Target_Tensor = torch.LongTensor(Target_Tensor)

  # Goal is to get to Trials X Neurons X Times for the neural data.
  HC_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TrainVal[i]]+'_KDHC.dat'), delimiter=',')
  HC_df = HC_df.values.reshape(HC_df.shape[0], int(HC_df.shape[1]/400), 400)
  HC_df = np.sum(HC_df,1) ##this is a way to avoid the conv layer.
  #try:
  #  HC_df = HC_df[:,np.random.choice(HC_df.shape[1], 13, replace = False),:]
  #except:
  #  break
  HC_Tensor = torch.FloatTensor(HC_df)

  PF_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TrainVal[i]]+'_KDPF.dat'), delimiter=',')
  PF_df = PF_df.values.reshape(PF_df.shape[0], int(PF_df.shape[1]/400), 400)
  PF_df = np.sum(PF_df,1)
  #PF_df = PF_df[:,np.random.choice(PF_df.shape[1], 13, replace = False),:]
  #try:
  #  PF_df = PF_df[:,np.random.choice(PF_df.shape[1], 13, replace = False),:]
  #except:
  #  break
  PF_Tensor = torch.FloatTensor(PF_df)

  loader = DataLoader(list(zip(PF_Tensor, Target_Tensor)), shuffle=True, batch_size=20)

  #Validation Data:
# For the regular data, the goal is just to get it into vector form.
  Val_Target_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TrainVal[-1]]+"_KD.csv"))
  Val_Target_Tensor = torch.tensor(Val_Target_df.values)
  Val_Target_Tensor = torch.LongTensor(Val_Target_Tensor)

  # Goal is to get to Trials X Neurons X Times for the neural data.
  Val_HC_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TrainVal[-1]]+'_KDHC.dat'), delimiter=',')
  Val_HC_df = Val_HC_df.values.reshape(Val_HC_df.shape[0], int(Val_HC_df.shape[1]/400), 400)
  Val_HC_df = np.sum(Val_HC_df,1) ##this is a way to avoid the conv layer.
  #try:
  #  Val_HC_df = Val_HC_df[:,np.random.choice(Val_HC_df.shape[1], 13, replace = False),:]
  #except:
  #  break
  #Val_HC_Tensor = torch.FloatTensor(Val_HC_df)

  Val_PF_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TrainVal[-1]]+'_KDPF.dat'), delimiter=',')
  Val_PF_df = Val_PF_df.values.reshape(Val_PF_df.shape[0], int(Val_PF_df.shape[1]/400), 400)
  Val_PF_df = np.sum(Val_PF_df,1)
  #PF_df = PF_df[:,np.random.choice(PF_df.shape[1], 13, replace = False),:]
  #try:
  #  Val_PF_df = Val_PF_df[:,np.random.choice(Val_PF_df.shape[1], 13, replace = False),:]
  #except:
  #  break
  Val_PF_Tensor = torch.FloatTensor(Val_PF_df)

  #Now begin with DNN.

  best_acc = 0
  TrackLoss = []
  TrackValLoss = []
  for i in range(0,5):
    model = Net(400,300,200,100,50,1) #was: 400,300,200,100,1
    # training parameters
    n_epochs = 180 #180 was the last one you tried that
    #loss_fn = torch.nn.PoissonNLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) #was .0001
    model.train()

    for epoch in range(n_epochs):
      for X_batch, y_batch in loader:
          y_pred = model(X_batch)
          loss = poisson_loss(y_pred, y_batch) #loss_fn to use built in loss. #50x31x1 pred vs. 50x1 batch ##torch.argmax(y_pred, 1) fixes the misalignment of sizes, but causes another issue: it isn't differentiable, so it has no gradient and will break the backward loss.
          TrackLoss.append(loss)
          optimizer.zero_grad()
          loss.backward(retain_graph=True) ##change this option?
          #scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.60, total_iters=30)....maybe eliminate b/c is not LR?
          optimizer.step() #add a tracker of the validation loss. You need to track both the training and val loss.

    model.eval()
    ###Now test on validation set:
    y_pred = model(Val_PF_Tensor)
    ce = poisson_loss(y_pred,Val_Target_Tensor)
    TrackValLoss.append(ce)
    current_acc = (torch.round(y_pred).int() == Val_Target_Tensor).float().mean()
    if current_acc > best_acc:
      best_acc = current_acc
      best_model = model

###Then Test the Best Performing Model.
TestAccList = [];
NIR_Num = np.array([]);
NIR_Den = np.array([]);
for i in range(0,len(TestVal)): #reserve last file in TrainVal for validation.
  # For the regular data, the goal is just to get it into vector form.
  Target_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TestVal[i]]+"_KD.csv"))
  Target_Tensor = torch.tensor(Target_df.values)
  Target_Tensor = torch.LongTensor(Target_Tensor)
  NIR_Num = np.append(NIR_Num,sum(Target_df.values==0))
  NIR_Den = np.append(NIR_Den,len(Target_df.values))

  # Goal is to get to Trials X Neurons X Times for the neural data.
  HC_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TestVal[i]]+'_KDHC.dat'), delimiter=',')
  HC_df = HC_df.values.reshape(HC_df.shape[0], int(HC_df.shape[1]/400), 400)
  HC_df = np.sum(HC_df,1) ##this is a way to avoid the conv layer.
  #HC_df = HC_df[:,np.random.choice(HC_df.shape[1], 13, replace = False),:]...downsample to match min.
  HC_Tensor = torch.FloatTensor(HC_df)

  PF_df = pd.read_csv(os.path.join(folder_path, PossibleFiles[TestVal[i]]+'_KDPF.dat'), delimiter=',')
  PF_df = PF_df.values.reshape(PF_df.shape[0], int(PF_df.shape[1]/400), 400)
  PF_df = np.sum(PF_df,1)
  #PF_df = PF_df[:,np.random.choice(PF_df.shape[1], 13, replace = False),:]
  PF_Tensor = torch.FloatTensor(PF_df)

  #test on training data for now, not actually test data:
  testy_pred = best_model(HC_Tensor) #adjusting this, and adjusting model(tensor) call will change between HC and PF
  test_acc = (torch.round(testy_pred).int() == Target_Tensor).float().mean()
  TestAccList.append(test_acc)

torch.stack(TestAccList).mean()  #average model performance. #best is: M=.53. SD=.11. Good news: it doesn't seem to predict all 0s or 1s.
torch.stack(TestAccList).std()

#confusion (one loop only, not averages):
Hits = torch.sum(torch.eq(Target_Tensor, torch.round(testy_pred).int() == 1)) #both 1
Miss = torch.sum(torch.eq(Target_Tensor, torch.round(testy_pred).int() != 1)) #true 1, pred 0
FA = torch.sum(torch.eq(1-Target_Tensor, torch.round(testy_pred).int() == 1)) #true 0, pred 1
CRs = torch.sum(torch.eq(1-Target_Tensor, torch.round(testy_pred).int() != 1)) #both 0

#NIR:
max(sum(NIR_Num)/sum(NIR_Den),1-sum(NIR_Num)/sum(NIR_Den))

##try the following: -->set a min of 38 neurons, this will exclude some sessions, but try that. --> Try the random subset of 13 neurons again. Try with 13.
#-->record the output with both of these options.
